<a href="https://colab.research.google.com/github/ShimonMalnick/ComputerVision/blob/master/Neuron_Visualization/Neuron_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Architecture
I use a pretrained [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), with architecture as shown below (credits to Krizhevsky et al.):

![AlexNet Architecture](src/images/alexnet_architecture.png)

# Construct The AlexNet Classifier
Note that like in the [Simonyan et al. 2014](https://arxiv.org/abs/1312.6034) paper, I've removed the softmax activation layer after the last fully connected layer.

In [ ]:
from __future__ import absolute_import, division, print_function, \
    unicode_literals
import numpy as np
from PIL import Image
import scipy
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Activation, \
    MaxPooling2D, Dropout
from tensorflow.keras import Model
import matplotlib.pyplot as plt

class AlexNet(Model):
    def __init__(self):
        super(AlexNet, self).__init__()
        # OPS
        self.relu = Activation('relu')
        self.maxpool = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                                    padding='valid')
        # self.dropout = Dropout(0.4)  # droput can is discarded for inference

        # Conv layers
        self.conv1 = Conv2D(filters=96, input_shape=(224, 224, 3),
                            kernel_size=(11, 11), strides=(4, 4),
                            padding='same')
        self.conv2a = Conv2D(filters=128, kernel_size=(5, 5), strides=(1, 1),
                             padding='same')
        self.conv2b = Conv2D(filters=128, kernel_size=(5, 5), strides=(1, 1),
                             padding='same')
        self.conv3 = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1),
                            padding='same')
        self.conv4a = Conv2D(filters=192, kernel_size=(3, 3), strides=(1, 1),
                             padding='same')
        self.conv4b = Conv2D(filters=192, kernel_size=(3, 3), strides=(1, 1),
                             padding='same')
        self.conv5a = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1),
                             padding='same')
        self.conv5b = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1),
                             padding='same')

        # Fully-connected layers

        self.flatten = Flatten()

        self.dense1 = Dense(4096, input_shape=(100,))
        self.dense2 = Dense(4096)
        self.dense3 = Dense(1000)

        # Network definition

    def call(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = tf.nn.local_response_normalization(x, depth_radius=2, alpha=2e-05,
                                               beta=0.75, bias=1.0)
        x = self.maxpool(x)

        x = tf.concat(
            (self.conv2a(x[:, :, :, :48]), self.conv2b(x[:, :, :, 48:])), 3)
        x = self.relu(x)
        x = tf.nn.local_response_normalization(x, depth_radius=2, alpha=2e-05,
                                               beta=0.75, bias=1.0)
        x = self.maxpool(x)

        x = self.conv3(x)
        x = self.relu(x)
        x = tf.concat(
            (self.conv4a(x[:, :, :, :192]), self.conv4b(x[:, :, :, 192:])), 3)
        x = self.relu(x)
        x = tf.concat(
            (self.conv5a(x[:, :, :, :192]), self.conv5b(x[:, :, :, 192:])), 3)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.flatten(x)

        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)

        return x


# Define The Training Process
Remember that instead of training the network and change its weights, we apply the gradients on the input image itself

In [ ]:
REGULARIZITAION_LAMBDA = np.float(0.05)

@tf.function
def train_steps(model, I, neuron_idx):
    with tf.GradientTape() as tape:
        predictionsI = model(I)
        score = predictionsI[0][neuron_idx]
        regularizer = tf.math.reduce_mean(tf.math.square(I)) * REGULARIZITAION_LAMBDA
        lossI = -(score - regularizer)
    gradientsI = tape.gradient(lossI, I)
    return gradientsI, lossI, score

optimizerI = tf.keras.optimizers.Adam()

def visualize_class(I, model, neuron_idx, number_of_iterations):

    for i in range(number_of_iterations):
        gradientI, lossI, score = train_steps(model, I, neuron_idx)
        optimizerI.apply_gradients([(gradientI, I)])
        cur_score = model(I)[0][neuron_idx]

    return I.numpy()

Mounted at /content/drive
tf.Tensor(0.015955525, shape=(), dtype=float32)
tf.Tensor(-0.9552753, shape=(), dtype=float32)
tf.Tensor(-0.97168684, shape=(), dtype=float32)


# Download The AlexNet Weights
the weights can be found [here](https://drive.google.com/drive/folders/1E2CIRBfo3rYTSDJOaNOVkLgfBpTjF9cB?usp=sharing), and should be placed in src/alexnet_weights

In [ ]:
import os

assert os.path.isdir('./src/alexnet_weights'), "Please download the alexnet weights from the link above"    

# Load The Weights

In [ ]:
I = tf.Variable(tf.random.uniform((1, 224, 224, 3)))
model = AlexNet()
model(I)

weights_dir = './src/alexnet_weights/'

model.conv1.set_weights(
    (np.load(weights_dir + 'conv1.npy'), np.load(weights_dir + 'conv1b.npy')))
model.conv2a.set_weights(
    (np.load(weights_dir + 'conv2_a.npy'), np.load(weights_dir + 'conv2b_a.npy')))
model.conv2b.set_weights(
    (np.load(weights_dir + 'conv2_b.npy'), np.load(weights_dir + 'conv2b_b.npy')))
model.conv3.set_weights(
    (np.load(weights_dir + 'conv3.npy'), np.load(weights_dir + 'conv3b.npy')))
model.conv4a.set_weights(
    (np.load(weights_dir + 'conv4_a.npy'), np.load(weights_dir + 'conv4b_a.npy')))
model.conv5a.set_weights(
    (np.load(weights_dir + 'conv5_a.npy'), np.load(weights_dir + 'conv5b_a.npy')))
model.conv4b.set_weights(
    (np.load(weights_dir + 'conv4_b.npy'), np.load(weights_dir + 'conv4b_b.npy')))
model.conv5b.set_weights(
    (np.load(weights_dir + 'conv5_b.npy'), np.load(weights_dir + 'conv5b_b.npy')))

model.dense1.set_weights(
    (np.load(weights_dir + 'dense1.npy'), np.load(weights_dir + 'dense1b.npy')))
model.dense2.set_weights(
    (np.load(weights_dir + 'dense2.npy'), np.load(weights_dir + 'dense2b.npy')))
model.dense3.set_weights(
    (np.load(weights_dir + 'dense3.npy'), np.load(weights_dir + 'dense3b.npy')))


# Optimize The Input Image
Here the neuron I try to activate corresponds to the score of the class of 'parachute'.
Classes ID's can be found in the classes.py file.

In [ ]:
PARACHUTE_CLASS_ID = 701
ITER_NUM = 20000

im = visualize_class(I, model, PARACHUTE_CLASS_ID, ITER_NUM)

#Normalizing the image for better results
def normalize_im(I):
  min = np.min(I, axis=(0, 1))
  max = np.max(I, axis=(0, 1))
  I = (I - min) / (max - min)
  return I

im = normalize_im(im[0])
im = np.clip(np.flip(im, 2), 0, 1)

# Show The Results

In [ ]:
plt.imshow(im)
plt.show()